In [1]:
import sys
sys.path.append("/home/jkdamewo/utils")

from imports import *
from django_imports import *
from datetime import datetime
import time
WORKDIR = os.getcwd()

In [2]:
group = Group.objects.get(name='HighEntropyAlloys')
config = JobConfig.objects.filter(name__in=['Vacancy_Relaxation_Oct29'])[0]

In [3]:
crys_parent = Crystal.objects.filter(
        parentjob__group=group,
        parentjob__config=config,
).distinct()

In [4]:
print(len(crys_parent))

4185


In [5]:
def traverse_unrelaxed_all_opt_jobs(parent_crystals, config_name='pbe_paw_opt_vasp', tqdm_disable=False):
    # Traverse from unrelaxed crystals to all of the most recent opt jobs
    
    children = []
    
    for parent in tqdm(parent_crystals, disable=tqdm_disable):
        
        found_most_recent = False            
        parent_cryst = parent
        
        while (not found_most_recent):
            
            try:
                childjob = list(parent_cryst.childjobs.filter(config__name=config_name).order_by('createtime'))[-1]
            except (Job.DoesNotExist, 
                    Geom.DoesNotExist,
                    django.core.exceptions.MultipleObjectsReturned,
                    IndexError):
                break
            
            if childjob.status != 'retried':                    
                found_most_recent = True
                children.append(childjob)
            else:
                try:
                    children.append(childjob)
                    parent_cryst = list(childjob.childgeoms.filter().order_by('id'))[-1].crystal
                except:
                    print("error")
                    break

    return children


def job_status_counter(jobs):
    # Count the statistic of job status

    counter = {}
    
    for job in jobs:
        job_status = job.status
        if job_status not in counter.keys():
            counter[job_status] = []
        counter[job_status].append(job)
    
    for key in counter.keys():
        print("Number of jobs in status", key, ":", len(counter[key]))
    
    return counter 

def traverse_opt_all_latest_es_jobs(parent_crystals, config_name, tqdm_disable=False):
    # Traverse from opt done and converged crystals to to all of the most recent es jobs

    children = []
    
    for parent in tqdm(parent_crystals, disable=tqdm_disable):

        found_most_recent = False            
        parent_cryst = parent

        while (not found_most_recent):
            
            try:
                childjob = list(parent_cryst.childjobs.filter(config__name=config_name).order_by('createtime'))[-1]
            except (Job.DoesNotExist, 
                    Geom.DoesNotExist,
                    django.core.exceptions.MultipleObjectsReturned):
                break
            
            found_most_recent = True
            children.append(childjob)

In [6]:
recent_jobs = traverse_unrelaxed_all_opt_jobs(crys_parent)

100%|██████████| 4185/4185 [00:16<00:00, 255.72it/s]


In [7]:
len(recent_jobs)

4059

In [8]:
count_failed = 0
correct_jobs = []
failed_jobs = []
for job in recent_jobs:
    if job.details["kpoints"]!=[3,3,3] or job.details["ismear"]!=1 or job.details["sigma"]!=0.2:
        count_failed+=1
        failed_jobs.append(job)
    else:
        correct_jobs.append(job)

In [9]:
len(correct_jobs)

3696

In [10]:
stored_dataset = {}

In [11]:
from pymatgen.core import Structure

In [12]:
import numpy as np

In [13]:
KBAR_TO_HARTREE_BOHR3 = 0.1/29421.02648438959

for i in range(len(correct_jobs)):
    if i % 10 == 0:
        print(i)
    curr_job = correct_jobs[i]
    stored_dataset[i] = {}
    curr_geoms = curr_job.childgeoms.filter()
    for iter_geom in range(len(curr_geoms)):
        curr_step = curr_geoms[iter_geom]
        stored_dataset[i][iter_geom] = {}
        try:
            energy = curr_step.calcs.filter(props__totalenergy__isnull=False)[0].props["totalenergy"]
            forces = np.asarray(curr_step.calcs.filter(props__stress__isnull=False)[0].jacobian.forces)
            stresses = np.asarray(curr_step.calcs.filter(props__stress__isnull=False)[0].props["stress"])
            stored_dataset[i][iter_geom]["structure"] = curr_step.crystal.as_pymatgen_structure().as_dict()
            stored_dataset[i][iter_geom]["energy_per_atom"] = (energy*27.2114) / 63.0
            stored_dataset[i][iter_geom]["force"] = (forces*51.4220861908).tolist()
            stored_dataset[i][iter_geom]["stress"] = (stresses*-1.0*(1/KBAR_TO_HARTREE_BOHR3)).tolist()
        except (Job.DoesNotExist, 
                    Geom.DoesNotExist,
                    django.core.exceptions.MultipleObjectsReturned,
                    IndexError):
            break
    

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
2010
2020
2030
2040
2050
2060
2070
2080
2090
2100
2110
2120
2130
2140
2150
2160
2170
2180
2190
2200
2210
2

In [14]:
all_count = 0
for key in stored_dataset:
    all_count += len(stored_dataset[key])

In [15]:
print(all_count)

67628


In [16]:
import json

with open("Vacancies_for_Myles_Jan_9.json", 'w') as file_object:  #open the file in write mode
    json.dump(stored_dataset, file_object) 